# Pinecone

>[Pinecone](https://docs.pinecone.io/docs/overview) is a vector database with broad functionality.

In the walkthrough, we'll demo the `SelfQueryRetriever` with a `Pinecone` vector store.

## Creating a Pinecone index
First we'll want to create a `Pinecone` vector store and seed it with some data. We've created a small demo set of documents that contain summaries of movies.

To use Pinecone, you have to have `pinecone` package installed and you must have an API key and an environment. Here are the [installation instructions](https://docs.pinecone.io/docs/quickstart).

**Note:** The self-query retriever requires you to have `lark` package installed.

In [1]:
%pip install --upgrade --quiet  lark

In [2]:
%pip install --upgrade --quiet pinecone-notebooks pinecone-client==3.2.2

In [3]:
# Connect to Pinecone and get an API key.
from pinecone_notebooks.colab import Authenticate

Authenticate()

import os

api_key = os.environ["PINECONE_API_KEY"]

We want to use `OpenAIEmbeddings` so we have to get the OpenAI API Key.

In [4]:
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [5]:
from pinecone import Pinecone, ServerlessSpec

api_key = os.getenv("PINECONE_API_KEY") or "PINECONE_API_KEY"

index_name = "langchain-self-retriever-demo"

pc = Pinecone(api_key=api_key)

In [6]:
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

embeddings = OpenAIEmbeddings()

# create new index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

In [7]:
docs = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"year": 1993, "rating": 7.7, "genre": ["action", "science fiction"]},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
    Document(
        page_content="Three men walk into the Zone, three men walk out of the Zone",
        metadata={
            "year": 1979,
            "director": "Andrei Tarkovsky",
            "genre": ["science fiction", "thriller"],
            "rating": 9.9,
        },
    ),
]
vectorstore = PineconeVectorStore.from_documents(
    docs, embeddings, index_name="langchain-self-retriever-demo"
)

## Creating our self-querying retriever
Now we can instantiate our retriever. To do this we'll need to provide some information upfront about the metadata fields that our documents support and a short description of the document contents.

In [8]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import OpenAI

metadata_field_info = [
    AttributeInfo(
        name="genre",
        description="The genre of the movie",
        type="string or list[string]",
    ),
    AttributeInfo(
        name="year",
        description="The year the movie was released",
        type="integer",
    ),
    AttributeInfo(
        name="director",
        description="The name of the movie director",
        type="string",
    ),
    AttributeInfo(
        name="rating", description="A 1-10 rating for the movie", type="float"
    ),
]
document_content_description = "Brief summary of a movie"
llm = OpenAI(temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm, vectorstore, document_content_description, metadata_field_info, verbose=True
)

## Testing it out
And now we can try actually using our retriever!

In [9]:
# This example only specifies a relevant query
retriever.invoke("What are some movies about dinosaurs")

[Document(page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose', metadata={'genre': ['action', 'science fiction'], 'rating': 7.7, 'year': 1993.0}),
 Document(page_content='Toys come alive and have a blast doing so', metadata={'genre': 'animated', 'year': 1995.0}),
 Document(page_content='Three men walk into the Zone, three men walk out of the Zone', metadata={'director': 'Andrei Tarkovsky', 'genre': ['science fiction', 'thriller'], 'rating': 9.9, 'year': 1979.0}),
 Document(page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea', metadata={'director': 'Satoshi Kon', 'rating': 8.6, 'year': 2006.0})]

In [10]:
# This example only specifies a filter
retriever.invoke("I want to watch a movie rated higher than 8.5")

[Document(page_content='Three men walk into the Zone, three men walk out of the Zone', metadata={'director': 'Andrei Tarkovsky', 'genre': ['science fiction', 'thriller'], 'rating': 9.9, 'year': 1979.0}),
 Document(page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea', metadata={'director': 'Satoshi Kon', 'rating': 8.6, 'year': 2006.0})]

In [11]:
# This example specifies a query and a filter
retriever.invoke("Has Greta Gerwig directed any movies about women")

[Document(page_content='A bunch of normal-sized women are supremely wholesome and some men pine after them', metadata={'director': 'Greta Gerwig', 'rating': 8.3, 'year': 2019.0})]

In [12]:
# This example specifies a composite filter
retriever.invoke("What's a highly rated (above 8.5) science fiction film?")

[Document(page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea', metadata={'director': 'Satoshi Kon', 'rating': 8.6, 'year': 2006.0}),
 Document(page_content='Three men walk into the Zone, three men walk out of the Zone', metadata={'director': 'Andrei Tarkovsky', 'genre': ['science fiction', 'thriller'], 'rating': 9.9, 'year': 1979.0})]

In [13]:
# This example specifies a query and composite filter
retriever.invoke(
    "What's a movie after 1990 but before 2005 that's all about toys, and preferably is animated"
)

[]

## Filter k

We can also use the self query retriever to specify `k`: the number of documents to fetch.

We can do this by passing `enable_limit=True` to the constructor.

In [14]:
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    enable_limit=True,
    verbose=True,
)

In [15]:
# This example only specifies a relevant query
retriever.invoke("What are two movies about dinosaurs")

[Document(page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose', metadata={'genre': ['action', 'science fiction'], 'rating': 7.7, 'year': 1993.0}),
 Document(page_content='Toys come alive and have a blast doing so', metadata={'genre': 'animated', 'year': 1995.0})]

## Using a hybrid vector store with self-query retriever

To perform hybrid search, specify a sparse encoder model when instantiating the vector store that we will use with the self query retriever.

For this demonstration we are using a fitted default `BM25Encoder` model from pinecone_text, `ChatOpenAI` as our llm model, and the same `OpenAIEmbeddings` embeddings model from before. 

In [16]:
from langchain_openai import ChatOpenAI
from pinecone_text.sparse import BM25Encoder

bm25_encoder = BM25Encoder().default()
model = ChatOpenAI(temperature=0)

To setup a hybrid index, the distance metric must be `dotproduct`. 

In [17]:
hybrid_index_name = "langchain-self-retriever-demo-hybrid"
distance_strategy = "dotproduct"

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]
if hybrid_index_name not in existing_indexes:
    pc.create_index(
        name=hybrid_index_name,
        dimension=1536,
        metric="dotproduct",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

Now we pass the necessary arguments that we have setup when instantiating the vector store.

In [18]:
hybrid_vector_store = PineconeVectorStore(
    embedding=embeddings,
    sparse_encoder=bm25_encoder,
    index_name=hybrid_index_name,
    distance_strategy=distance_strategy,
)

To perform upserts into the index, we can use the `add_texts` function. For texts, we will use the `page_content` portion of the pre-split docs from before.

Since we have passed an encoder model alongside the embeddings model, the upserts have added both sparse and dense values for each text added.

In [19]:
hybrid_vector_store.add_texts(
    texts=[doc.page_content for doc in docs], metadatas=[doc.metadata for doc in docs]
)

['6d0bc4c5-3ae8-47f2-ab6c-f73c477e35f4',
 '4c743336-315a-44a5-983b-79221c5d9113',
 'c5c8eada-d978-4ef6-9847-f606c7c00b40',
 '26883c54-0dd8-4963-a3e8-11a06c45be95',
 '8bf1a0b5-2df7-47bf-9579-aad045770209',
 '3ec1a25e-4109-414a-8282-c238b50e09d3']

Next, we set up a query_constructor for the self query retriever. Here we use `load_query_constructor_runnable`. 

In [20]:
from langchain.chains.query_constructor.base import load_query_constructor_runnable
from langchain_pinecone._utilities import allowed_comparators, allowed_operators

query_constructor_runnable = load_query_constructor_runnable(
    llm=model,
    document_contents=document_content_description,
    attribute_info=metadata_field_info,
    allowed_comparators=allowed_comparators,
    allowed_operators=allowed_operators,
    fix_invalid=True,
)

<!-- We also need to use `PineconeTranslator`?, which contains pinecone index specific requirements and constraints. -->

In [21]:
hybrid_retriever = SelfQueryRetriever(
    query_constructor=query_constructor_runnable,
    vectorstore=hybrid_vector_store,
    fix_invalid=True,
    enable_limit=True,
)

Define a test question and view the resulting structured query.

In [22]:
question = "What's a movie that's about dream dream dream dreaming about a hazy zone of dreaming, with a rating above 1?"

In [23]:
query_constructor_runnable.invoke(question)

StructuredQuery(query='dream hazy zone', filter=Comparison(comparator=<Comparator.GT: 'gt'>, attribute='rating', value=1), limit=None)

A purely semantic search

In [24]:
hybrid_retriever.search_kwargs = {"k": 2, "alpha": 1}
hybrid_retriever.invoke({"query": question})

[Document(page_content='Three men walk into the Zone, three men walk out of the Zone', metadata={'director': 'Andrei Tarkovsky', 'genre': ['science fiction', 'thriller'], 'rating': 9.9, 'year': 1979.0}),
 Document(page_content='Leo DiCaprio gets lost in a dream within a dream within a dream within a ...', metadata={'director': 'Christopher Nolan', 'rating': 8.2, 'year': 2010.0})]

A purely keyword search

In [25]:
hybrid_retriever.search_kwargs = {"k": 2, "alpha": 0}  # keyword search
hybrid_retriever.invoke({"query": question})

[Document(page_content='Leo DiCaprio gets lost in a dream within a dream within a dream within a ...', metadata={'director': 'Christopher Nolan', 'rating': 8.2, 'year': 2010.0}),
 Document(page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea', metadata={'director': 'Satoshi Kon', 'rating': 8.6, 'year': 2006.0})]

Let's try another question, this time with `alpha=0.5` to see hybrid search results in comparison to keyword search.

In [26]:
query_constructor_runnable.invoke("What are movies about scientists")

StructuredQuery(query='scientists', filter=None, limit=None)

In [27]:
hybrid_retriever.search_kwargs = {"k": 2, "alpha": 0.5}
hybrid_retriever.invoke({"query": "What are movies about scientists"})

[Document(page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose', metadata={'genre': ['action', 'science fiction'], 'rating': 7.7, 'year': 1993.0}),
 Document(page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea', metadata={'director': 'Satoshi Kon', 'rating': 8.6, 'year': 2006.0})]

In [28]:
hybrid_retriever.search_kwargs = {"k": 2, "alpha": 0}
hybrid_retriever.invoke({"query": "What are movies about scientists"})

[Document(page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose', metadata={'genre': ['action', 'science fiction'], 'rating': 7.7, 'year': 1993.0}),
 Document(page_content='A bunch of normal-sized women are supremely wholesome and some men pine after them', metadata={'director': 'Greta Gerwig', 'rating': 8.3, 'year': 2019.0})]